In [1]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [3]:
ethnea_df = pd.read_csv('names_ethnea_genni_country.csv')
#ethnea_df

In [4]:
# First thing first, use the character feature for making the dnn model
def extract_structure(word,n_char=2):
    x_struct = []
    word_len = len(word) + n_char
    n_char-=1
    counter = 0
    for i in range(word_len):
        end = i+1
        start = (i - n_char) if (i - n_char) > 0 else 0
        if word[start:end]!='_' and word[start:end]!='':
        #if word[start:end]!='_':
            x_struct.append(word[start:end])
    return x_struct

first_name_struct = ethnea_df.First.apply(lambda x: extract_structure(x.lower(),2))
last_name_struct = ethnea_df.Last.apply(lambda x: extract_structure(x.lower(),2))                                                                

In [5]:
# make struct dictionary
struct_dict = {}
for name_struct_i in first_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1
for name_struct_i in last_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1

In [6]:
struct_dict_keys = list(struct_dict.keys())
ethnic_series = ethnea_df['Ethnea'].str.lower()
ethnic_keys = list(np.unique(ethnic_series.values))

In [7]:
len(first_name_struct)

4434085

In [8]:
# load test train data
with open('train_test_full_index.pickle', 'rb') as f:
    trainIndex,testIndex,trainY,testY = pickle.load(f)

In [9]:
# transform the dataset into structure
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

In [10]:
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

def generate_batch(first_name, last_name, i, batch_size=10000):
    len_name = len(first_name)
    start = i*batch_size
    end = start+batch_size if start+batch_size < len_name else len_name
    len_mat = end - start
    #first_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    #last_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    first_name_ds_mat = first_name[start:end]
    last_name_ds_mat = last_name[start:end]
    first_name_ds_mat = pad_sequences(first_name_ds_mat.apply(lambda x: transform_structure(x)),maxlen=50,value=0.)
    first_name_ds_mat = first_name_ds_mat.reshape(first_name_ds_mat.shape[0],1,first_name_ds_mat.shape[1])
    last_name_ds_mat = pad_sequences(last_name_ds_mat.apply(lambda x:transform_structure(x)),maxlen=50,value=0.)
    last_name_ds_mat = last_name_ds_mat.reshape(last_name_ds_mat.shape[0],1,last_name_ds_mat.shape[1])

    """
    for i in range(len_mat):
        x = first_name_struct.iloc[i]
        for y in x:
            first_name_ds_mat[i,struct_dict_keys.index(y)]+=1
        x = last_name_struct.iloc[i]
        for y in x:
            last_name_ds_mat[i,struct_dict_keys.index(y)]+=1
    """
    return first_name_ds_mat,last_name_ds_mat, range(start,end)

In [40]:
list(range(5,10))

[5, 6, 7, 8, 9]

In [11]:
def transform_labels(x):
    y = np.zeros(len(ethnic_keys))
    y[ethnic_keys.index(x)]=1
    return y

labels = np.array(list(map(lambda x: transform_labels(x),ethnic_series)))

In [12]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tflearn.data_utils import to_categorical, pad_sequences



Using TensorFlow backend.


In [13]:
from keras.layers import Input
from keras.models import Model

"""
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
"""

# max sequence length
seq_length = 100
# multi input with single output

# first name input
first_name_input = Input(shape=(1,50),name='first_name_input')
last_name_input = Input(shape=(1,50),name='last_name_input')

# first tensor for first name
first_name_l = Bidirectional(LSTM(1000,return_sequences=False))(first_name_input)
last_name_l = Bidirectional(LSTM(1000,return_sequences=False))(last_name_input)

# merge the two layer together
x = keras.layers.concatenate([first_name_l,last_name_l])

# stack dense network for memory
x = Dense(1000, activation='relu')(x)
x = Dense(500, activation='relu')(x)
output_l = Dense(len(ethnic_keys),activation='softmax')(x)

model = Model(inputs=[first_name_input, last_name_input], outputs=[output_l])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 2000)          8408000     first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 2000)          8408000     last_name_input[0][0]            
___________________________________________________________________________________________

In [15]:
model.summary()
first_trainX = first_name_struct[trainIndex]
first_testX = first_name_struct[testIndex]
last_trainX = last_name_struct[trainIndex]
last_testX = last_name_struct[testIndex]
trainY = labels[trainIndex]
testY = labels[testIndex]

#trainX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in trainX])
#testX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in testX])

mini_batch_size = 50000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 10000

for x in range(1):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        model.fit([y_first_trainX, y_last_trainX],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 2000)          8408000     first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 2000)          8408000     last_name_input[0][0]            
___________________________________________________________________________________________

50000/50000 [==============================] - 103s - loss: 0.6788 - acc: 0.8047    
Epoch 1/1
50000/50000 [==============================] - 102s - loss: 0.6862 - acc: 0.8024    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.6915 - acc: 0.7992    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.6718 - acc: 0.8068     
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.6766 - acc: 0.8046     
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.6794 - acc: 0.8043    
Epoch 1/1
50000/50000 [==============================] - 102s - loss: 0.6684 - acc: 0.8059    
Epoch 1/1
47268/47268 [==============================] - 92s - loss: 0.6666 - acc: 0.8078     


In [16]:
mini_batch_size = 50000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(10):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        model.fit([y_first_trainX, y_last_trainX],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.6869 - acc: 0.7985    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.7242 - acc: 0.7912    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.7330 - acc: 0.7849    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.7387 - acc: 0.7873    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.7156 - acc: 0.7924    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.7249 - acc: 0.7904    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.7248 - acc: 0.7880    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.7209 - acc: 0.7913    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.7126 - acc: 0.7935    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.7066 - acc: 0.7933    
Epoch 1/1
50000/50000 [==============================] - 98

50000/50000 [==============================] - 100s - loss: 0.5448 - acc: 0.8404    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.5423 - acc: 0.8398    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.5441 - acc: 0.8404    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.5487 - acc: 0.8384    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.5367 - acc: 0.8423    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.5402 - acc: 0.8411    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.5478 - acc: 0.8384    
Epoch 1/1
50000/50000 [==============================] - 102s - loss: 0.5518 - acc: 0.8376    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.5469 - acc: 0.8404    
Epoch 1/1
50000/50000 [==============================] - 102s - loss: 0.5452 - acc: 0.8408    
Epoch 1/1
50000/50000 [==============================] - 101

50000/50000 [==============================] - 94s - loss: 0.4726 - acc: 0.8590    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.4736 - acc: 0.8590    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4744 - acc: 0.8594    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4719 - acc: 0.8588    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4694 - acc: 0.8601    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4644 - acc: 0.8617    
Epoch 1/1
50000/50000 [==============================] - 94s - loss: 0.4753 - acc: 0.8589    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4682 - acc: 0.8609    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4679 - acc: 0.8616    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4610 - acc: 0.8624    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0

50000/50000 [==============================] - 95s - loss: 0.4268 - acc: 0.8731    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4247 - acc: 0.8720    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4316 - acc: 0.8696    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4264 - acc: 0.8707    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4246 - acc: 0.8727    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.4206 - acc: 0.8732    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.4217 - acc: 0.8719    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.4216 - acc: 0.8728    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.4187 - acc: 0.8736    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.4234 - acc: 0.8718    
Epoch 1/1
50000/50000 [==============================] - 99s - loss:

50000/50000 [==============================] - 95s - loss: 0.3959 - acc: 0.8804    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3843 - acc: 0.8841    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3880 - acc: 0.8815    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3923 - acc: 0.8807    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3864 - acc: 0.8817    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3901 - acc: 0.8821    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3903 - acc: 0.8811    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3874 - acc: 0.8824    
Epoch 1/1
47268/47268 [==============================] - 90s - loss: 0.3877 - acc: 0.8816    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3945 - acc: 0.8789    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0

50000/50000 [==============================] - 97s - loss: 0.3608 - acc: 0.8882    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3645 - acc: 0.8872    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3606 - acc: 0.8900    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3639 - acc: 0.8883    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3634 - acc: 0.8894    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.3635 - acc: 0.8879    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3594 - acc: 0.8917    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3669 - acc: 0.8882    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3585 - acc: 0.8905    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3602 - acc: 0.8883    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0

50000/50000 [==============================] - 95s - loss: 0.3396 - acc: 0.8939    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3440 - acc: 0.8941    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3342 - acc: 0.8957    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3432 - acc: 0.8943    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3400 - acc: 0.8947    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3357 - acc: 0.8965    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.3404 - acc: 0.8951    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.3314 - acc: 0.8971    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.3359 - acc: 0.8977    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.3373 - acc: 0.8974    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 

50000/50000 [==============================] - 96s - loss: 0.3165 - acc: 0.9014    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3140 - acc: 0.9009    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3154 - acc: 0.9022    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3178 - acc: 0.9018    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3186 - acc: 0.9011    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3210 - acc: 0.8992    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3074 - acc: 0.9047    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3178 - acc: 0.9009    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3221 - acc: 0.8996    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3184 - acc: 0.9010    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0

50000/50000 [==============================] - 96s - loss: 0.3043 - acc: 0.9051    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.3024 - acc: 0.9063    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2978 - acc: 0.9072    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3046 - acc: 0.9054    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3023 - acc: 0.9052    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2974 - acc: 0.9066    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2959 - acc: 0.9070    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.3044 - acc: 0.9060    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2997 - acc: 0.9056    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.3035 - acc: 0.9043    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0

In [39]:
mini_batch_size = 50000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(5):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        model.fit([y_first_trainX, y_last_trainX],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.2957 - acc: 0.9076    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.3004 - acc: 0.9047    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2948 - acc: 0.9045    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.3050 - acc: 0.9047    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.3027 - acc: 0.9058    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.2935 - acc: 0.9069     
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.2974 - acc: 0.9055    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.2941 - acc: 0.9072    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2918 - acc: 0.9078    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2939 - acc: 0.9073   
Epoch 1/1
50000/50000 [=============================

50000/50000 [==============================] - 96s - loss: 0.2789 - acc: 0.9125    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2773 - acc: 0.9136    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2826 - acc: 0.9100    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2744 - acc: 0.9131    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2750 - acc: 0.9133    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2795 - acc: 0.9118    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2849 - acc: 0.9099    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2808 - acc: 0.9098    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2842 - acc: 0.9098    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2768 - acc: 0.9117    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0

50000/50000 [==============================] - 97s - loss: 0.2711 - acc: 0.9141    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.2697 - acc: 0.9134     
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2655 - acc: 0.9153    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2571 - acc: 0.9185    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2736 - acc: 0.9131    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.2678 - acc: 0.9139    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2663 - acc: 0.9151    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2643 - acc: 0.9150    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2632 - acc: 0.9164    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2647 - acc: 0.9155    
Epoch 1/1
50000/50000 [==============================] - 97s - loss:

50000/50000 [==============================] - 96s - loss: 0.2554 - acc: 0.9174    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2520 - acc: 0.9198    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2484 - acc: 0.9206    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2545 - acc: 0.9181    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2532 - acc: 0.9190    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2530 - acc: 0.9195    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2536 - acc: 0.9195    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2474 - acc: 0.9221    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2500 - acc: 0.9197    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2541 - acc: 0.9190    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0

50000/50000 [==============================] - 98s - loss: 0.2429 - acc: 0.9219    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2414 - acc: 0.9224    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2413 - acc: 0.9213    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2423 - acc: 0.9226    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2466 - acc: 0.9204    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2380 - acc: 0.9231    
Epoch 1/1
47268/47268 [==============================] - 93s - loss: 0.2345 - acc: 0.9242    


In [42]:
mini_batch_size = 50000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(5):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        model.fit([y_first_trainX, y_last_trainX],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
50000/50000 [==============================] - 110s - loss: 0.2412 - acc: 0.9226    
Epoch 1/1
50000/50000 [==============================] - 106s - loss: 0.2449 - acc: 0.9214    
Epoch 1/1
50000/50000 [==============================] - 105s - loss: 0.2456 - acc: 0.9197    
Epoch 1/1
50000/50000 [==============================] - 113s - loss: 0.2440 - acc: 0.9214    
Epoch 1/1
50000/50000 [==============================] - 112s - loss: 0.2426 - acc: 0.9220    
Epoch 1/1
50000/50000 [==============================] - 116s - loss: 0.2375 - acc: 0.9225    
Epoch 1/1
50000/50000 [==============================] - 119s - loss: 0.2381 - acc: 0.9226    
Epoch 1/1
50000/50000 [==============================] - 119s - loss: 0.2387 - acc: 0.9227    
Epoch 1/1
50000/50000 [==============================] - 118s - loss: 0.2403 - acc: 0.9220    
Epoch 1/1
50000/50000 [==============================] - 118s - loss: 0.2372 - acc: 0.9230    
Epoch 1/1
50000/50000 [===========================

50000/50000 [==============================] - 97s - loss: 0.2302 - acc: 0.9249    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2278 - acc: 0.9261    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2279 - acc: 0.9256    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2283 - acc: 0.9260    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2261 - acc: 0.9274    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2273 - acc: 0.9257    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2272 - acc: 0.9266    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2327 - acc: 0.9243    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2291 - acc: 0.9252    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2286 - acc: 0.9254    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0

50000/50000 [==============================] - 96s - loss: 0.2178 - acc: 0.9283    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2210 - acc: 0.9282    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2238 - acc: 0.9274    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2208 - acc: 0.9272    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2161 - acc: 0.9280    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2254 - acc: 0.9270    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2229 - acc: 0.9266    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2208 - acc: 0.9281    
Epoch 1/1
50000/50000 [==============================] - 98s - loss: 0.2211 - acc: 0.9282    
Epoch 1/1
50000/50000 [==============================] - 97s - loss: 0.2158 - acc: 0.9291    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 

50000/50000 [==============================] - 95s - loss: 0.2173 - acc: 0.9295    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0.2191 - acc: 0.9284    
Epoch 1/1
50000/50000 [==============================] - 94s - loss: 0.2094 - acc: 0.9305    
Epoch 1/1
50000/50000 [==============================] - 94s - loss: 0.2102 - acc: 0.9312    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2145 - acc: 0.9292    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2108 - acc: 0.9298    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2091 - acc: 0.9311    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2086 - acc: 0.9308    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2091 - acc: 0.9325    
Epoch 1/1
50000/50000 [==============================] - 95s - loss: 0.2079 - acc: 0.9311    
Epoch 1/1
50000/50000 [==============================] - 96s - loss: 0

50000/50000 [==============================] - 99s - loss: 0.2020 - acc: 0.9326    
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.1985 - acc: 0.9354    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1994 - acc: 0.9339    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.2033 - acc: 0.9326    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.2009 - acc: 0.9334     
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.2031 - acc: 0.9321    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2057 - acc: 0.9308   
Epoch 1/1
50000/50000 [==============================] - 101s - loss: 0.1981 - acc: 0.9337    
Epoch 1/1
47268/47268 [==============================] - 94s - loss: 0.1927 - acc: 0.9364    


In [ ]:
mini_batch_size = 50000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(5):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        model.fit([y_first_trainX, y_last_trainX],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
50000/50000 [==============================] - 113s - loss: 0.2039 - acc: 0.9322    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2045 - acc: 0.9321    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2056 - acc: 0.9310    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2057 - acc: 0.9312    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2060 - acc: 0.9311    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2007 - acc: 0.9326    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2015 - acc: 0.9325    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2011 - acc: 0.9327    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.1966 - acc: 0.9362    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1999 - acc: 0.9338    
Epoch 1/1
50000/50000 [============================

50000/50000 [==============================] - 100s - loss: 0.1933 - acc: 0.9347    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1905 - acc: 0.9365    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1898 - acc: 0.9367    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1938 - acc: 0.9349    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1903 - acc: 0.9373    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1887 - acc: 0.9367    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1933 - acc: 0.9363    
Epoch 1/1
50000/50000 [==============================] - 102s - loss: 0.1932 - acc: 0.9353    
Epoch 1/1
50000/50000 [==============================] - 99s - loss: 0.1933 - acc: 0.9351    
Epoch 1/1
50000/50000 [==============================] - 100s - loss: 0.2008 - acc: 0.9331    
Epoch 1/1
50000/50000 [==============================] - 100s - los

50000/50000 [==============================] - 103s - loss: 0.1889 - acc: 0.9367    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1893 - acc: 0.9376    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1910 - acc: 0.9371    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1880 - acc: 0.9372    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1876 - acc: 0.9376    
Epoch 1/1
50000/50000 [==============================] - 102s - loss: 0.1838 - acc: 0.9386    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1926 - acc: 0.9368    
Epoch 1/1
50000/50000 [==============================] - 104s - loss: 0.1886 - acc: 0.9378    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1836 - acc: 0.9403    
Epoch 1/1
50000/50000 [==============================] - 103s - loss: 0.1891 - acc: 0.9368    
Epoch 1/1
50000/50000 [==============================] - 103

In [43]:
# save this model 
model_json = model.to_json()
with open('model-keras-w3-bigram-full-lstm.json','w') as f:
    f.write(model_json)
#save the last weight
model.save_weights('model-keras-w3-bigram-full-lstm-25.h5')    

In [37]:
# evaluation
def evaluate(firstX, lastX, testY, testIndex, mini_batch_size=20000):
    test_len = len(testY)
    len_mini_batch = round(test_len/mini_batch_size)
    scores = np.zeros(len_mini_batch)
    for i in range(len_mini_batch):
        y_first_trainX,y_last_trainX,batch_range = generate_batch(firstX,lastX,i,mini_batch_size)
        #pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys), pub_series[testIndex].iloc[batch_range])))
        scores[i] = model.evaluate([y_first_trainX,y_last_trainX],testY[batch_range])[1]
        print(scores[i])
    return np.average(scores)
    #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
    

In [41]:
evaluate(first_testX,last_testX, testY, testIndex)

20000/20000 [==============================] - 34s    
0.8822
20000/20000 [==============================] - 35s    
0.88435
20000/20000 [==============================] - 34s    
0.88005
20000/20000 [==============================] - 34s    
0.88335
20000/20000 [==============================] - 34s    
0.8802
20000/20000 [==============================] - 34s    
0.8852
20000/20000 [==============================] - 32s    
0.8844
20000/20000 [==============================] - 32s    
0.88195
20000/20000 [==============================] - 32s    
0.8852
20000/20000 [==============================] - 33s    
0.8839
20000/20000 [==============================] - 33s    
0.882
20000/20000 [==============================] - 32s    
0.8843
20000/20000 [==============================] - 32s    
0.8808
20000/20000 [==============================] - 30s    
0.8818
20000/20000 [==============================] - 30s    
0.8827
20000/20000 [==============================] - 34s    
0.8783
20000

0.88314772727272739

In [22]:
def trans_name(name):
    name = name.lower()
    # transform space into underscore
    name = '_'+name.replace(' ','_')+'_'
    #transform the name into sequence structure
    ext_name = extract_structure(name)
    trans_name = transform_structure(ext_name)
    """
    name_ds_mat = np.zeros((1,len(struct_dict_keys)),dtype=np.int32)
    for i,x in enumerate(trans_name):
        name_ds_mat[0,x-1]+=1
    """
    trans_name = pad_sequences([trans_name], maxlen=50,value=0.)
    trans_name = trans_name.reshape(trans_name.shape[0],1,trans_name.shape[1])
    return trans_name

def predict_ethnicity(fname,lname):
    # lower case the name
    fnamex = trans_name(fname)
    lnamex = trans_name(lname)
    pred = model.predict([np.array(fnamex),np.array(lnamex)])
    pred_class = np.argsort(pred[0])[::-1]
    return_item = []
    for x in np.argsort(pred[0])[::-1]:
        return_item.append((ethnic_keys[x],pred[0][x]))
    return return_item

In [34]:
predict_ethnicity('Swathi','Namburi')
#trans_name('Nikolaus')

[('indian', 0.98119122),
 ('hispanic', 0.01019792),
 ('english', 0.0038582406),
 ('arab', 0.0021652307),
 ('italian', 0.0017684674),
 ('israeli', 0.00066581945),
 ('german', 9.2379858e-05),
 ('japanese', 1.511315e-05),
 ('slav', 8.5945176e-06),
 ('french', 7.4365657e-06),
 ('thai', 6.0584334e-06),
 ('greek', 5.5856085e-06),
 ('caribbean', 5.4785969e-06),
 ('nordic', 3.0941119e-06),
 ('african', 3.0229498e-06),
 ('chinese', 2.2395416e-06),
 ('dutch', 2.2289325e-06),
 ('polynesian', 5.9792535e-07),
 ('romanian', 5.3214677e-07),
 ('hungarian', 4.3925164e-07),
 ('indonesian', 1.3866767e-07),
 ('mongolian', 1.0278641e-07),
 ('baltic', 4.923513e-08),
 ('korean', 8.505797e-09),
 ('turkish', 3.9258587e-09),
 ('vietnamese', 3.454189e-11)]

In [8]:
# embedd the structure vocabulary using text embedding and reduce the dimensionality

# convert the names into word structure vector
struct_dict_keys = list(struct_dict.keys())

def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

In [9]:
def transform_labels(x):
    y = np.zeros(len(ethnic_keys))
    y[ethnic_keys.index(x)]=1
    return y

labels = np.array(list(map(lambda x: transform_labels(x),ethnic_series)))

In [26]:
# using tflearn make the graph creation simple
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split

# separate train and training set
trainX, testX, trainY, testY = train_test_split(data_source,[ethnic_keys.index(x) for x in ethnic_series],test_size = 0.2)

trainX = pad_sequences(trainX, maxlen=50,value=0.)
testX = pad_sequences(testX,maxlen=50,value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY,nb_classes=len(ethnic_keys))
testY = to_categorical(testY,nb_classes=len(ethnic_keys))    

In [16]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import pickle

with open('train_test_fix.pickle', 'rb') as f:
    trainX,trainY,testX,testY,ethnic_keys,struct_dict_keys = pickle.load(f)
    #aha = pickle.load(f)

#with open('traintest-smote.pickle','rb') as f:
#    train_res,test_res = pickle.load(f)

with open('ethnic_keys.pickle','rb') as f:
    name_struct_keys,ethnic_keys = pickle.load(f)
        
embedding_vector_length = 1000
lstm_layer = 1000
max_sequence = 50

In [ ]:
# convert categorical to binary crossentropy
#trainY = np.array([np.where(x>0)[0][0] for x in trainY])
#testY = np.array([np.where(x>0)[0][0] for x in testY])

#test_res

In [2]:
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
for x in range(10):
    model.fit(trainX,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 1000)          62696000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 1000)          3001000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               880800    
_________________________________________________________________
dense_1 (Dense)              (None, 23)                4623      
Total params: 66,582,423
Trainable params: 66,582,423
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
35653/35653 [==============================] - 778s - loss: 1.6879 - acc: 0.5265    
Accuracy: 75.88%
Epoch 1/1
35653/35653 [==============================] - 796s - loss: 0.5197 - acc: 0.8582    
Accuracy: 84.37%
Epoch 1/1
35653/35653 [==========

In [4]:
testX

array([[  112,  1331,  1332, ...,     0,     0,     0],
       [  251,   252,   232, ...,     0,     0,     0],
       [ 4633, 11731, 23123, ...,     0,     0,     0],
       ..., 
       [  696,   697,   698, ...,     0,     0,     0],
       [   25,    26,  5195, ...,     0,     0,     0],
       [ 8671,  8672,  6568, ...,     0,     0,     0]], dtype=int32)

In [12]:
from keras.models import model_from_json
from keras import backend as K

# compute the accuracy
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2
    #print(precision)

    # How many relevant items are selected?
    recall = c1 / c3
    #print(recall)

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    return precision


def recall(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    recall = c1 / c3

    return recall

# load model
# load json and create model
json_file = open('model-keras-embed-bilstm-womaxpool.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#                               ,custom_objects= {'f1_score': f1_score})
loaded_model.load_weights("model-keras-embed-bilstm-womaxpool-10.h5")

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',f1_score,precision,recall])

In [18]:
scores = loaded_model.evaluate(testX,testY,verbose=0)

In [19]:
print('Accuracy: {}, F1: {}, Precision: {}, Recal: {}'.format(scores[1],scores[2],scores[3],scores[4]))

Accuracy: 0.8561812878216707, F1: 0.8601400889915978, Precision: 0.8757469038779302, Recal: 0.845523895023381


In [181]:
from tflearn.data_utils import to_categorical, pad_sequences

# transform prediction
# given name compute the prediction
def predict_ethnicity(name):
    # lower case the name
    name = name.lower()
    # transform space into underscore
    name = '_'+name.replace(' ','_')+'_'
    #transform the name into sequence structure
    ext_name = extract_structure(name)
    trans_name = transform_structure(ext_name)
    trans_name = pad_sequences([trans_name], maxlen=50,value=0.)
    pred = loaded_model.predict(trans_name)
    pred_class = np.argsort(pred[0])[::-1]
    return_item = []
    for x in np.argsort(pred[0])[::-1]:
        return_item.append((ethnic_keys[x],pred[0][x]))
    return return_item

name='helen lamothe'
ext_name = extract_structure(name)
#print(ext_name)
trans_name = transform_structure(ext_name)
#trans_name
pad_sequences([trans_name], maxlen=50,value=0.)
#extract_structure('Nikolaus Nova')
#transform_structure('Robert Nova')
ethnic_prob = predict_ethnicity('Filho  Elias Abdalla')
#ethnic_prob

In [132]:
test = loaded_model.predict(trainX[10].reshape(1,50))
test

array([[  1.22544670e-03,   2.45175033e-05,   5.54050894e-06,
          3.56604069e-05,   1.31730601e-04,   1.58663862e-03,
          9.95586514e-01,   3.65224201e-04,   4.18016425e-05,
          1.64734403e-04,   3.29397808e-05,   1.42851750e-05,
          5.77516516e-07,   3.13429664e-05,   1.98912196e-04,
          5.65968139e-06,   4.63458673e-06,   4.45792568e-04,
          5.28864875e-05,   2.15678101e-05,   8.53615438e-06,
          4.57901763e-07,   1.44505730e-05]], dtype=float32)

In [133]:
np.argsort(test)[0][::-1]

array([ 6,  5,  0, 17,  7, 14,  9,  4, 18,  8,  3, 10, 13,  1, 19, 22, 11,
       20, 15,  2, 16, 12, 21])

In [134]:
np.where(trainY[10]==1)

(array([6]),)

In [151]:
name='_helen__lamothe_'
ext_name = extract_structure(name)
#print(ext_name)
trans_name = transform_structure(ext_name)
#trans_name
pad_sequences([trans_name], maxlen=50,value=0.)

array([[    0,     0,     0,  2304,  2305,     0,     0,     0,     0,
         4088, 15559,  2523, 15102,     0,     0,     0,    24,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]], dtype=int32)

In [191]:
predict_ethnicity('harry potter')

[('ITALIAN', 0.52735686),
 ('INDIAN', 0.30269179),
 ('KOREAN', 0.097788125),
 ('JAPANESE', 0.017470013),
 ('ROMANIAN', 0.015210837),
 ('TURKISH', 0.014220745),
 ('HISPANIC', 0.0092350421),
 ('GERMAN', 0.0046770978),
 ('BALTIC', 0.0032188322),
 ('ARAB', 0.0028918688),
 ('ISRAELI', 0.0022467086),
 ('GREEK', 0.0014537659),
 ('SLAV', 0.00046977124),
 ('NORDIC', 0.00024971511),
 ('HUNGARIAN', 0.0002173665),
 ('DUTCH', 0.00021363674),
 ('INDONESIAN', 0.00019762212),
 ('VIETNAMESE', 9.9613972e-05),
 ('AFRICAN', 3.4369114e-05),
 ('CHINESE', 3.3702971e-05),
 ('ENGLISH', 1.7328795e-05),
 ('FRENCH', 4.1550811e-06),
 ('THAI', 9.4588233e-07)]

In [175]:
ethnic_keys[15]

'ISRAELI'

In [71]:
ethnea_df['First']+ethnea_df['Last']+ethnea_df['Ethnea']

0                           _Elias_Abdalla__Filho_HISPANIC
1                                  _Jad__Bou_Abdallah_ARAB
2                                  _Ayman__Abdel_Aziz_ARAB
3                                  _Salma__Abdelmoula_ARAB
4                                     _Ibrahim__Abdou_ARAB
5                             _Hazem__Abou_El_Fettouh_ARAB
6                                     _Rola__Aboutaam_ARAB
7        _Aida_Alexandra__Alvim_de_Abreu_Silva_Rodrigue...
8             _Isabel__Cristina_Affonso_Scaletsky_HISPANIC
9                                _Tsiri__Agbenyega_AFRICAN
10                     _Jose__Maria_Aguado_Garcia_HISPANIC
11                      _Manuela__Aguilar_Guisado_HISPANIC
12                         _Arturo__Aguillon_Luna_HISPANIC
13                                   _Ali__Ahmadzadeh_ARAB
14                                    _Ahmed__Ibrahim_ARAB
15                                    _Ahmed__Letaief_ARAB
16                           _Mohammed__Shakeel_Ahmed_AR